In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-04-02 18:45:32.283636


In [2]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 
pd.set_option("display.max.columns", None) 
 
url = "https://litteraturbanken.se/api/get_authors"
r = http.request('GET', url)
data = json.loads(r.data)
dfLitt = pd.json_normalize(data["data"])

In [3]:
dfLittShowTrue = dfLitt[dfLitt["show"] == True].copy()
dfLittShowTrue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3790 entries, 2 to 4882
Data columns (total 50 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   authorid                       3790 non-null   object 
 1   authorid_norm                  3790 non-null   object 
 2   db_checksum                    3790 non-null   object 
 3   db_timestamp                   3790 non-null   int64  
 4   doc_type                       3790 non-null   object 
 5   full_name                      3790 non-null   object 
 6   gender                         3790 non-null   object 
 7   imported                       3361 non-null   object 
 8   intro                          723 non-null    object 
 9   name_for_index                 3790 non-null   object 
 10  pictureinfo                    247 non-null    object 
 11  searchable                     3790 non-null   bool   
 12  show                           3790 non-null   b

In [4]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys,json
import pandas as pd 

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

# 
query = """SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID)  ?authorid ?SBL ?SKBL WHERE {
?item wdt:P31 wd:Q5.
?item wdt:P5101 ?authorid
OPTIONAL {?item wdt:P3217 ?SBL}
OPTIONAL {?item wdt:P4963 ?SKBL}
} order by ?authorid"""
 
    
def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']
    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

WDLittbanktot = get_sparql_dataframe(endpoint_url, query)

In [5]:
WDLittbanktot

,WikidataID,authorid,SBL,SKBL
0,Q11967131,AasenE,None,None
1,Q4933592,AbeniusM,None,MargitAbenius
2,Q24680938,AbrahamssonA,None,None
3,Q4934135,AbrahamssonS,None,None
4,Q365923,AchariusE,5503,None
...,...,...,...,...
3412,Q108406266,ÖstergrenC,None,None
3413,Q6257795,ÖstergrenCL,None,None
3414,Q100752816,ÖstinO,None,None
3415,Q6258216,ÖstmanK,None,None


In [6]:
# df = pd.merge(dfA, dfB, on=['a','b'], how="outer", indicator=True)
WDLittbank_WD_merge=  pd.merge(dfLittShowTrue,WDLittbanktot, how="outer", on='authorid',indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge_Outer"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge_Outer"].value_counts()



both          3266
left_only      524
right_only     151
Name: WD_Littbank_merge_Outer, dtype: int64

In [7]:
WDLittbank_WD_merge_Not_connected = \
WDLittbank_WD_merge[WDLittbank_WD_merge["WD_Littbank_merge_Outer"] == "left_only"].copy()

In [8]:
WDLittbank_WD_merge_Not_connected.head()

,authorid,authorid_norm,db_checksum,db_timestamp,doc_type,full_name,gender,imported,intro,name_for_index,pictureinfo,searchable,show,surname,updated,birth.date,birth.plain,death.date,death.plain,librisid,wikidata.birthplace,wikidata.birthplace_label,wikidata.deathplace,wikidata.deathplace_label,wikidata.image,wikidata.sbl_link,wikidata.skbl_link,wikidata.sol_link,wikidata.wikidata_id,wikidata.wikipedia,db_timestamp_updated,intro_text,popularity,pseudonym,dramawebben.intro,dramawebben.intro_author,dramawebben.intro_author_norm,dramawebben.legacy_url,dramawebben.picture,sources,other_name,intro_author,intro_author_norm,dramawebben.picture_info,picture,bibliography,external_ref,presentation,seemore,dramawebben.sources,WikidataID,SBL,SKBL,WD_Littbank_merge_Outer
7,Addäos,Addaos,da5e35f5f2e32602a17344e44b2b6e93,1.679916e+12,author,Addäos,male,2014-08-22,None,Addäos,None,False,True,Addäos,2016-04-12,-350,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
30,AhlbergL,AhlbergL,ee16e9ff206c29114ebb85c687f26a3d,1.679916e+12,author,Ludvig Ahlberg,male,2021-03-01,None,"Ahlberg, Ludvig",None,True,True,Ahlberg,2021-03-01,NaN,1809,1888,1888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679916e+12,NaN,67.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
39,AhlmanC,AhlmanC,ded5b5ffeea552a3c1ea1778d4430364,1.679916e+12,author,Carl O. Ahlman,male,2019-02-25,None,"Ahlman, Carl O.",None,False,True,Ahlman,2019-02-25,1830,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679916e+12,NaN,344.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
49,AhrenbergB,AhrenbergB,b1741fb810e253a036639d61e7a3563f,1.679916e+12,author,Bertha Ahrenberg,female,2022-01-31,None,"Ahrenberg, Bertha",None,True,True,Ahrenberg,2022-01-31,1851,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679916e+12,NaN,826.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
66,AllvarssonA,AllvarssonA,3dfce487ff899fd930a9216c072f4f21,1.679916e+12,author,Anders Allvarsson,male,2023-03-27,None,"Allvarsson, Anders",None,True,True,Allvarsson,2023-03-27,1871,0000,NaN,0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.679916e+12,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [9]:
WDLittbank_WD_merge_Not_connected[WDLittbank_WD_merge_Not_connected['librisid']. 
                                  notna()][["authorid","librisid","show","full_name"]]

,authorid,librisid,show,full_name
93,AminoffI,20dgcrwl4vl10b2,True,Ivan Aminoff
139,AnholmM,b8nrv6rv4frp22g,True,Maria Anholm
157,ArbmanE,pm1344x720qnbbc,True,Eva Arbman
163,ArfwidssonN,hftwzq01397wbbd,True,Nils Arfwidsson
247,BeckmanS,mkz1xpx53hwbhqk,True,Susen Beckman
276,BergAH,tr58dmrc4717v4g,True,Anders Henrik Berg
902,EngelbrechtFG,c9ps1dcw1nskq19,True,Fredrik Gustaf Engelbrecht
926,EricsonA,fcrtwgtz5tq1nr9,True,Alfred Ericson
1086,FryxellDP,b8nqlvfv3cz5xjs,True,Daniel Peter Fryxell
1094,FröléenJF,vs68bxbd3zn1p80,True,Johan Fredrik Fröléen


In [10]:
WDLittbank_WD_merge_Not_connected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 524 entries, 7 to 3787
Data columns (total 54 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   authorid                       524 non-null    object  
 1   authorid_norm                  524 non-null    object  
 2   db_checksum                    524 non-null    object  
 3   db_timestamp                   524 non-null    float64 
 4   doc_type                       524 non-null    object  
 5   full_name                      524 non-null    object  
 6   gender                         524 non-null    object  
 7   imported                       402 non-null    object  
 8   intro                          21 non-null     object  
 9   name_for_index                 524 non-null    object  
 10  pictureinfo                    1 non-null      object  
 11  searchable                     524 non-null    object  
 12  show                           524 

In [11]:
WDLittbank_WD_merge_Not_connected["librisid"].value_counts()

20dgcrwl4vl10b2     1
b8nrv6rv4frp22g     1
pm1344x720qnbbc     1
hftwzq01397wbbd     1
mkz1xpx53hwbhqk     1
tr58dmrc4717v4g     1
c9ps1dcw1nskq19     1
fcrtwgtz5tq1nr9     1
b8nqlvfv3cz5xjs     1
vs68bxbd3zn1p80     1
0xbdfjvj5rjnrc0     1
0xbfjxsj53jsrk0     1
wt7bgwtf3zrkx8c     1
blrms1168p9xphh0    1
gdsvzxn032qd1hm     1
Name: librisid, dtype: int64

In [12]:
WDLittbank_WD_merge_Not_connected[WDLittbank_WD_merge_Not_connected["librisid"] =="20dgkckl01lmd80"]

,authorid,authorid_norm,db_checksum,db_timestamp,doc_type,full_name,gender,imported,intro,name_for_index,pictureinfo,searchable,show,surname,updated,birth.date,birth.plain,death.date,death.plain,librisid,wikidata.birthplace,wikidata.birthplace_label,wikidata.deathplace,wikidata.deathplace_label,wikidata.image,wikidata.sbl_link,wikidata.skbl_link,wikidata.sol_link,wikidata.wikidata_id,wikidata.wikipedia,db_timestamp_updated,intro_text,popularity,pseudonym,dramawebben.intro,dramawebben.intro_author,dramawebben.intro_author_norm,dramawebben.legacy_url,dramawebben.picture,sources,other_name,intro_author,intro_author_norm,dramawebben.picture_info,picture,bibliography,external_ref,presentation,seemore,dramawebben.sources,WikidataID,SBL,SKBL,WD_Littbank_merge_Outer


In [13]:
WDLittbank_WD_merge_Not_connected["wikidata.sbl_link"].value_counts()

Series([], Name: wikidata.sbl_link, dtype: int64)

In [14]:
#WDLittbank_WD_merge_Not_connected["sources"].value_counts()

In [15]:
#WDLittbank_WD_merge_Not_connected["popularity"].value_counts()

In [16]:
# WDLittbank_WD_merge_Not_connected["intro_text"].value_counts()